# Twitter Detection Suicide

In [ ]:
# Librairies to install
!pip install vaderSentiment
!pip install textblob
!pip install happytransformer
!pip install transformers

In [ ]:
# Important librairy
import pandas as pd
import numpy as np
import nltk
import os
import re
import warnings
warnings.filterwarnings('ignore')

# For Modele
from nltk.tag import UnigramTagger
from nltk.corpus import treebank
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from happytransformer import HappyTextClassification
from transformers import AutoModel, AutoTokenizer
from transformers import pipeline

# Analyse des données
import seaborn as sn
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix

In [ ]:
# On charge les deux dataset de test et de train
df = pd.read_csv('./Suicide_Detection_Reddit2.csv')
#df = pd.read_csv('./Suicide_Detection_Tweeter.csv')
#df = pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Suicide_Detection_Tweeter.csv')
df = df.rename(columns={
    "Tweet": "text",
    "Suicide": "class"
})
df["text"] = df["text"].astype(str)
df["class"] = df["class"].astype(str)
df.head(10)

In [ ]:
# On affiche la dimension du dataset
print('Nombre de ligne : ' +  str(df.shape[0]))
print('Nombre de colonne : ' + str(df.shape[1]))

# Cleaning

In [ ]:
def Cleaning(Text):
    #supprimer les identifiactions avec les @
    L=[]#liste des mots à enlever
    for i in range(len(Text)):
        if Text[i]=='@':            
            s=''
            for j in Text[i:]:
                if j != ' ':
                    s+=j
                else:
                    break            
            L.append(s)
    #suppression de tous les mots
    for word in L:
        Text=Text.replace(word,'')
    
    #On garde que les caractères alphanumériques
    Text=re.sub(r'((?![a-z]|[A-Z]|\d).)',' ',Text)#((?!...).) => opposé
    
    return Text.strip() #suppression des espaces au début et à la fin

In [ ]:
df['text'] = df['text'].apply(lambda x: Cleaning(x))
df.head(10)

# Modeles

## Fonction Globale

In [ ]:
# Instanciez le modèle et le tokeniseur
def Classificator(name):
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModel.from_pretrained(name)
    return pipeline(model=name)

In [ ]:
# Keyword Analyse
train_sents = treebank.tagged_sents()[:3000]
tagger = UnigramTagger(train_sents)

# Classificateur
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)
classifier_bert = Classificator("nlptown/bert-base-multilingual-uncased-sentiment")
classifier_suicidal_bert = Classificator("gooohjy/suicidal-bert")

In [ ]:
def AddModeleIntoDataframe(df, function_modele, list_column_name):
    df[list_column_name] = df['text'].apply(lambda x: function_modele(x))
    return df

In [ ]:
def AddAllModelToDF(df):
    #cleaning du dataframe
    df["text"] = df["text"].apply(lambda x: x[:500])
    
    print("Réalisation de la fonction keyword_analyse en cours ....")
    #df = AddModeleIntoDataframe(df, sentiment_keyword, "keyword_label")
    
    print("Réalisation de la fonction sentiment_vader en cours ....")
    df = AddModeleIntoDataframe(df, sentiment_vader, "valder_label")
    
    print("Réalisation de la fonction sentiment_texblob en cours ....")
    df = AddModeleIntoDataframe(df, sentiment_texblob_polarity, "textbloc_polarity_label")
    df = AddModeleIntoDataframe(df, sentiment_texblob_subjectivity, "textbloc_subjectivity_label")
    
    print("Réalisation de la fonction sentiment_happy_transformer en cours ....")
    df = AddModeleIntoDataframe(df, sentiment_happy_transformer, "happy_transformer_label")
    
    print("Réalisation de la fonction sentiment_bert en cours ....")
    df = AddModeleIntoDataframe(df, sentiment_bert, "bert_label")
    
    print("Réalisation de la fonction sentiment_suicidal_bert en cours ....")
    df = AddModeleIntoDataframe(df, sentiment_suicidal_bert, "suicidal_bert_label")

    return df

## KeyWord Analyse

In [ ]:
def sentiment_keyword(text):
    texte=word_tokenize(text)
    L=set()
    tag=tagger.tag(texte)

    for t in tag:
        L.add(t[0])

    pos_score=0
    neg_score=0

    for adv in L:
        adv_senti = list(swn.senti_synsets(adv))
        if len(adv_senti)!=0:
            pos_score+=adv_senti[0].pos_score()
            neg_score+=adv_senti[0].neg_score()
    res=''
    
    if pos_score > neg_score:
        res='pos'
    elif pos_score < neg_score:
        res='neg'

    return res,pos_score,neg_score

## Valder

In [ ]:
def sentiment_vader(sentence):
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"
        
    elif sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"   

    else :
        overall_sentiment = "Neutral"
  
    return overall_sentiment

## TextBlob

In [ ]:
def sentiment_texblob_polarity(row):
    classifier = TextBlob(row)
    polarity = classifier.sentiment.polarity
    return polarity

def sentiment_texblob_subjectivity(row):
    classifier = TextBlob(row)
    subjectivity = classifier.sentiment.subjectivity
    return subjectivity

## Happy Transformer

In [ ]:
def sentiment_happy_transformer(row):
    result = happy_tc.classify_text(row)
    return str(result.label)

## BERT

In [ ]:
def sentiment_bert(row):
    # On recupere le sentiment
    res = classifier_bert(row)

    # Ajoutez les résultats aux tableaux
    return res[0]['label'][:1]

## Suicidal BERT

In [ ]:
def sentiment_suicidal_bert(row):
    # On recupere le sentiment
    res = classifier_suicidal_bert(row)

    # Ajoutez les résultats aux tableaux
    return res[0]['label']

In [ ]:
df.head()

# Analyse

In [ ]:
def PreparationModel(df):
    df["class"] = df["class"].replace({"Potential Suicide post ": 1, "Not Suicide post": 0})
    df["valder_label"] = df["valder_label"].replace({"Negative": 1, "Positive": 0, "Neutral": 0})
    df["happy_transformer_label"] = df["happy_transformer_label"].replace({"NEGATIVE": 1, "POSITIVE": 0})
    df["bert_label"] = df["bert_label"].replace({"1": 1, "2": 0, "3": 0, "4": 0, "5": 0})
    df["suicidal_bert_label"] = df["suicidal_bert_label"].replace({"LABEL_1": 1, "LABEL_0": 0})
    return df

In [ ]:
df = AddAllModelToDF(df)
df = PreparationModel(df)
df.head()

In [ ]:
# Nous allons nous concentrez uniquement sur la caractéristique cible
plt.figure(figsize=(20,20))
heatmap = sn.heatmap(df.corr()[['class']].sort_values(by='class', ascending=False), vmin=-0.5, vmax=1, annot=True, cmap='Blues')
heatmap.set_title('Correlation du résultats avec les autres features', pad=16, fontdict={'family': 'serif','size': 20});

In [ ]:
#df.to_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Resultats_Owen.csv', sep=';', index=False)
#df.to_csv('./Suicide_Detection_Tweeter_Final.csv', index=False)
df.to_csv('./Suicide_Detection_Reddit_Final.csv', index=False, ,sep=';')

# Prediction

In [ ]:
df = pd.read_csv('./Suicide_Detection_Tweeter_Final.csv',sep=';')
# df = pd.read_csv('./Suicide_Detection_Reddit_Final.csv')

In [ ]:
def TauxBonneReponse(df):
    # Sélection des lignes où la colonne "class" contient la valeur 1
    selected_true_positive = df.loc[df['class'] == 1]
    return [selected_true_positive.shape[0], df.shape[0], round(selected_true_positive.shape[0] / df.shape[0], 2)]

In [ ]:
# Genere un dataframe ou stock le meilleur modele pour chaque méthode
ResultDf = pd.DataFrame()
def StockResultDf(name, df):
    global ResultDf
    tab = TauxBonneReponse(df)
    listValue = {'Methode Name': name, 'Nombre de bonne reponse': tab[0], 'Nombre de réponse totale': tab[1] , 'Taux de bonne réponse (%)': tab[2]}
    ResultDf = ResultDf.append(listValue,ignore_index=True)

In [ ]:
def SelectionTweetSuicidaire(df):
    #Methode Naif
    StockResultDf("Naif", df)
    
    #Methode 1
    # On ne garde que suicidal bert
    df1 = df[df['suicidal_bert_label'] == 1]
    StockResultDf("Model 1", df1)
    
    #Methode 2
    # Sélection des lignes où au moins 3 des colonnes ont la valeur 1
    df2 = df[(df[['valder_label', 'happy_transformer_label', 'bert_label', 'suicidal_bert_label']].sum(axis=1) >= 3)]
    StockResultDf("Model 2", df2)
    
#On execute puis on affiche le resultat
SelectionTweetSuicidaire(df.copy())
ResultDf

# Matrice de confusion

In [ ]:
df_init=pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Resultats_Owen.csv',sep=';')
df=df_init.copy()
df.head()

In [ ]:
df_init=pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Suicide_Detection_Reddit_Final.csv')
df=df_init.copy()
df.head()

In [ ]:
#Valder
matrix_valder = confusion_matrix(y_true=df['class'], y_pred=df['valder_label'])
matrix_valder = matrix_valder/(np.sum(matrix_valder))
df_matrix_valder = pd.DataFrame(matrix_valder, ['negatif','positif'], ['négatif','positif'])

#Happy transformer
matrix_hf = confusion_matrix(y_true=df['class'], y_pred=df['happy_transformer_label'])
matrix_hf = matrix_hf/(np.sum(matrix_hf))
df_matrix_hf = pd.DataFrame(matrix_hf, ['negatif','positif'], ['négatif','positif'])

#Bert
matrix_bert = confusion_matrix(y_true=df['class'], y_pred=df['bert_label'])
matrix_bert = matrix_bert/(np.sum(matrix_bert))
df_matrix_bert = pd.DataFrame(matrix_bert, ['negatif','positif'], ['négatif','positif'])

#Suicidal Bert
matrix_sbert = confusion_matrix(y_true=df['class'], y_pred=df['suicidal_bert_label'])
matrix_sbert = matrix_sbert/(np.sum(matrix_sbert))
df_matrix_sbert = pd.DataFrame(matrix_sbert, ['negatif','positif'], ['négatif','positif'])

In [ ]:
cmap=plt.cm.Blues

fig, axs = plt.subplots(2, 2 ,figsize=(8,8))
fig.suptitle('Matrices de confusion des modèles', fontsize=16)

#Valder
g1=sn.heatmap(df_matrix_valder, cmap=cmap, annot=True, fmt='.2%', cbar=False, vmin=0.2, vmax=0.6, ax=axs[0,0])
axs[0,0].set_title('Valder')
g1.set_ylabel('Labels')

#Happy transormer
g2=sn.heatmap(df_matrix_hf, cmap=cmap, annot=True, fmt='.2%',cbar=False, vmin=0.2, vmax=0.6, ax=axs[1,0])
axs[1,0].set_title('Happy transformer')
g2.set_xlabel('Prédictions')
g2.set_ylabel('Labels')

#Bert
g3=sn.heatmap(df_matrix_bert, cmap=cmap, annot=True, fmt='.2%', vmin=0.2, vmax=0.6, ax=axs[0,1])
axs[0,1].set_title('Bert')
#g2.set_xlabel('Prédictions')
#g2.set_ylabel('Labels')

#Suicidal Bert
g4=sn.heatmap(df_matrix_sbert, cmap=cmap, annot=True, fmt='.2%', vmin=0.2, vmax=0.6, ax=axs[1,1])
axs[1,1].set_title('Suicidal Bert')
g4.set_xlabel('Prédictions')
#g2.set_ylabel('Labels')

fig.tight_layout()
plt.show()

